In [ ]:
!pip3 install sentence-transformers gensim SPARQLWrapper
!pip install funcy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
from sentence_transformers import SentenceTransformer
from funcy import print_durations

encoder_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# def get_sentence_transformer_model(model_name):
#     model = SentenceTransformer(model_name_or_path=model_name)
#     return model

@print_durations
def get_embeddings(labels, sent_tran_model):
    embeddings = sent_tran_model.encode(labels, show_progress_bar=False)
    return embeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def retrieve_tbox(lang='en', offset=0):
    sparql = SPARQLWrapper('http://dbpedia.org/sparql')
    query = f"""
    SELECT ?uri ?label {{
      ?uri a ?type ; rdfs:label ?label .
      values(?type) {{ (owl:Class) (rdf:Property) }}
      filter(lang(?label) = '{lang}' && regex(?uri, "http://dbpedia.org/ontology/"))
    }} LIMIT 10000 OFFSET {offset}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    tbox = {}
    for result in results['results']['bindings']:
        uri = result['uri']['value']
        label = result['label']['value']
        if label not in tbox:
            tbox[label] = set()
        tbox[label].add(uri)
    return tbox


def get_labels_and_tbox():
    offset = 0
    tbox = {}
    while True:
        tbox_chunk = retrieve_tbox(lang='en', offset=offset)
        if len(tbox_chunk) == 0:
            break
        offset += 10000
        for k, v in tbox_chunk.items():
            if k not in tbox:
                tbox[k] = set()
            tbox[k] = tbox[k].union(v)
    labels = [l.replace('\n', ' ') for l in tbox]
    return labels, tbox

In [ ]:
def write_embeddings_to_file(embeddings, labels, filename):
    with open(filename, 'w', encoding='utf-8') as f_out:
        f_out.write(f"{len(labels)} {len(embeddings[0])}\n")
        for label, embedding in zip(labels, embeddings):
            f_out.write(f"{label.replace(' ', '_')} {' '.join([str(x) for x in embedding])}\n")
    print("Embeddings written to file successfully")

In [ ]:
from gensim.models import KeyedVectors

def load_gensim_model_from_file(filepath):
    model = KeyedVectors.load_word2vec_format(filepath, binary=False)
    return model

In [ ]:
import time
start = time.time()

labels, tbox = get_labels_and_tbox()
embeddings = get_embeddings(labels, encoder_model)
write_embeddings_to_file(embeddings, labels, "dbpedia-ontology.vectors")

end = time.time()
print(f"Time taken to compute and write embeddings - {end-start} seconds")

   12.89 s in get_embeddings(['location', 'Relates ..., SentenceTransformer( (...)
Embeddings written to file successfully
Time taken to compute and write embeddings - 21.802759647369385 seconds


In [ ]:
NS_RESOURCE = 'http://dbpedia.org/resource/'
NS_RESOURCE_LEN = len(NS_RESOURCE)

NS_ONTOLOGY = 'http://dbpedia.org/ontology/'
NS_ONTOLOGY_LEN = len(NS_ONTOLOGY)

def to_uri(label, tbox):
    return list(filter(lambda x: 'A' <= x[NS_ONTOLOGY_LEN : NS_ONTOLOGY_LEN+1] <= 'z', tbox[label]))

In [ ]:
import pandas as pd
import numpy as np

def dbpedia_similarity_search(pred, gensim_model, sent_tran_model, tbox, cluster_centers, embeddings, cluster_labels, threshold = 0.5):
    print("Predicting:", pred)
    pred_embedding = sent_tran_model.encode([pred], show_progress_bar=False)[0]

    pred_embedding = pred_embedding.reshape(-1)

    similarities = gensim_model.cosine_similarities(pred_embedding, cluster_centers)
    closest_cluster_idx = np.argmax(similarities)
    closest_similarity = similarities[closest_cluster_idx]

    ####****Threshold not determined yet****####

    # if closest_similarity < threshold:
    #   print(f"No close cluster found for predicate '{pred}' with similarity above {threshold}.")
    #   return ""


    print(f"Closest cluster index: {closest_cluster_idx}")
    print(f"Similarity to closest cluster: {closest_similarity}")

    group_indices = np.where(cluster_labels == closest_cluster_idx)[0]
    group_embeddings = [embeddings[i] for i in group_indices]
    group_labels = [labels[i] for i in group_indices]

    # print("Labels in the closest cluster:", group_labels)

    result = gensim_model.most_similar(positive=[pred_embedding], topn=1)
    out = []

    for label, score in result:
        out.append({'label': label.replace('_', ' '), 'score': score})
    df = pd.DataFrame(out)
    df.insert(0, 'URIs', df['label'].map(lambda x: to_uri(x, tbox=tbox)))
    return df


In [ ]:
from sklearn.cluster import KMeans


def cluster_embeddings(embeddings, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(embeddings)
    return kmeans.cluster_centers_, kmeans.labels_

In [ ]:
def write_cluster_centers_to_file(cluster_centers, filename):
    with open(filename, 'w', encoding='utf-8') as f_out:
        f_out.write(f"{len(cluster_centers)} {len(cluster_centers[0])}\n")
        for i, center in enumerate(cluster_centers):
            f_out.write(f"cluster_{i} {' '.join([str(x) for x in center])}\n")
    print("Cluster centers written to file successfully")

In [ ]:
cluster_centers, cluster_labels = cluster_embeddings(embeddings)
write_cluster_centers_to_file(cluster_centers, "dbpedia-ontology-cluster.vectors")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster centers written to file successfully


In [ ]:
# pred = "offspring of"
# pred = "child of"
# pred = "located at"
pred = "Shared border with"
# pred = "date of birth"

gensim_model = load_gensim_model_from_file("dbpedia-ontology.vectors")

db_pred = dbpedia_similarity_search(pred, gensim_model, encoder_model, tbox, cluster_centers, embeddings, cluster_labels)

print(db_pred)

Predicting: Shared border with
Closest cluster index: 3
Similarity to closest cluster: 0.2848406136035919
                                   URIs   label     score
0  [http://dbpedia.org/ontology/border]  border  0.759235
